# Защита данных клиентов страховой компании

<h1>Содержание<span class="tocSkip"></span></h1>
<div class="toc"><ul class="toc-item"><li><span><a href="#Загрузка-данных" data-toc-modified-id="Загрузка-данных-1"><span class="toc-item-num">1&nbsp;&nbsp;</span>Загрузка данных</a></span></li><li><span><a href="#Умножение-матриц" data-toc-modified-id="Умножение-матриц-2"><span class="toc-item-num">2&nbsp;&nbsp;</span>Умножение матриц</a></span></li><li><span><a href="#Алгоритм-преобразования" data-toc-modified-id="Алгоритм-преобразования-3"><span class="toc-item-num">3&nbsp;&nbsp;</span>Алгоритм преобразования</a></span></li><li><span><a href="#Проверка-алгоритма" data-toc-modified-id="Проверка-алгоритма-4"><span class="toc-item-num">4&nbsp;&nbsp;</span>Проверка алгоритма</a></span></li><li><span><a href="#Чек-лист-проверки" data-toc-modified-id="Чек-лист-проверки-5"><span class="toc-item-num">5&nbsp;&nbsp;</span>Чек-лист проверки</a></span></li></ul></div>

## Описание проекта

Вам нужно защитить данные клиентов страховой компании «Хоть потоп». Разработайте такой метод преобразования данных, чтобы по ним было сложно восстановить персональную информацию. Обоснуйте корректность его работы.

Нужно защитить данные, чтобы при преобразовании качество моделей машинного обучения не ухудшилось. Подбирать наилучшую модель не требуется.

Инструкция по выполнению проекта

   - Загрузите и изучите данные.
   - Ответьте на вопрос и обоснуйте решение.
   - Признаки умножают на обратимую матрицу. Изменится ли качество линейной регрессии? (Её можно обучить заново.)
     - a. Изменится. Приведите примеры матриц.
     - b. Не изменится. Укажите, как связаны параметры линейной регрессии в исходной задаче и в преобразованной.
   - Предложите алгоритм преобразования данных для решения задачи. Обоснуйте, почему качество линейной регрессии не поменяется.
   - Запрограммируйте этот алгоритм, применив матричные операции. Проверьте, что качество линейной регрессии из sklearn не отличается до и после преобразования. Примените метрику R2.

**Описание данных**

Набор данных находится в файле insurance.csv.

    Признаки: пол, возраст и зарплата застрахованного, количество членов его семьи.
    Целевой признак: количество страховых выплат клиенту за последние 5 лет.


In [1]:
# импорт необходимых библиотек
import pandas as pd
import numpy as np

from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler
from sklearn.linear_model import LinearRegression
from sklearn.metrics import r2_score

## Загрузка данных

In [2]:
# загрузка данных
try:
    insurance = pd.read_csv('data-files/insurance.csv')  # Локальный путь
except:
    insurance = pd.read_csv('/datasets/insurance.csv') # Серверный путь

In [3]:
insurance.head()

,Пол,Возраст,Зарплата,Члены семьи,Страховые выплаты
0,1,41.0,49600.0,1,0
1,0,46.0,38000.0,1,1
2,0,29.0,21000.0,0,0
3,0,21.0,41700.0,2,0
4,1,28.0,26100.0,0,0


In [4]:
insurance.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 5000 entries, 0 to 4999
Data columns (total 5 columns):
 #   Column             Non-Null Count  Dtype  
---  ------             --------------  -----  
 0   Пол                5000 non-null   int64  
 1   Возраст            5000 non-null   float64
 2   Зарплата           5000 non-null   float64
 3   Члены семьи        5000 non-null   int64  
 4   Страховые выплаты  5000 non-null   int64  
dtypes: float64(2), int64(3)
memory usage: 195.4 KB


In [5]:
print('Количество явных дубликатов:',insurance.duplicated().sum())

Количество явных дубликатов: 153


In [6]:
insurance.describe()

,Пол,Возраст,Зарплата,Члены семьи,Страховые выплаты
count,5000.000000,5000.000000,5000.000000,5000.000000,5000.000000
mean,0.499000,30.952800,39916.360000,1.194200,0.148000
std,0.500049,8.440807,9900.083569,1.091387,0.463183
min,0.000000,18.000000,5300.000000,0.000000,0.000000
25%,0.000000,24.000000,33300.000000,0.000000,0.000000
50%,0.000000,30.000000,40200.000000,1.000000,0.000000
75%,1.000000,37.000000,46600.000000,2.000000,0.000000
max,1.000000,65.000000,79000.000000,6.000000,5.000000


**Вывод**

Имеются набора данных о клиентах страховой компании: в наборе 5 000 клиентов. Для каждого клиента есть информация о поле, возрасте, зарплате, членах семьи и страховых выплатах.

Данные чистые, без аномалий и не требуют предобработки.
Проверка показывает наличие дубликатов. Однако, так как в данных нет уникального идентификатора клиента, это могут быть просто совпадения. Поэтому мнимые дубликаты не удалялись.


## Умножение матриц

**Ответьте на вопрос и обоснуйте решение.**

Признаки умножают на обратимую матрицу. Изменится ли качество линейной регрессии? (Её можно обучить заново.)

    a. Изменится. Приведите примеры матриц.
    b. Не изменится. Укажите, как связаны параметры линейной регрессии в исходной задаче и в преобразованной.

Обозначения:

- $X$ — матрица признаков (нулевой столбец состоит из единиц)

- $y$ — вектор целевого признака

- $P$ — матрица, на которую умножаются признаки

- $w$ — вектор весов линейной регрессии (нулевой элемент равен сдвигу)

Предсказания:

$$
a = Xw
$$

Задача обучения:

$$
w = \arg\min_w MSE(Xw, y)
$$

Формула обучения:

$$
w = (X^T X)^{-1} X^T y
$$

В случае умножение признаков на обратимую матрицу получим новую формулу обучения:

$$
w' = ((XP)^T XP)^{-1} (XP)^T y
$$

Проведем математические преобразования

$$
w' = ((XP)^T XP)^{-1} (XP)^T y = \\
= (P^T X^T X P)^{-1} (XP)^T y = \\
= ((P^T X^T X) P)^{-1} (XP)^T y = \\
= P^{-1} (P^T X^T X)^{-1} P^T X^T y = \\
= P^{-1} (X^T X)^{-1} (P^T)^{-1} P^T X^T y = \\
= P^{-1} (X^T X)^{-1} E X^T y = \\
= P^{-1} (X^T X)^{-1} X^T y = \\
= P^{-1} w
$$

То есть вектор параметров линейной регресси для преобразованной задачи равен матрице обратной P умноженной на вектор параметров для исходной задачи.

$$
w' =  P^{-1} w
$$

Сответсвенно, формула предсказаний для преобразованной задачи имеет вид:

$$
a' = XPw'
$$

После математических преобразований получим:
$$
a' = X P w' = X P P^{-1} w = X E w = Xw = a
$$

То есть предсказания для преобразованной и исходной задачи одинаковы.

**Ответ:**  При умножении матрицы признаков на обратимую матрицу качество линейной регрессии не изменится. Предсказания для преобразованной и исходной задачи одинаковы, а вектор параметров линейной регресси для преобразованной задачи равен обратной матрице P умноженной на вектор параметров для исходной задачи.


## Алгоритм преобразования

**Алгоритм**

Преобразование данных можно провести используя умножение признаков на обратимую матрицу и уже на этих данных обучить модель. Матрицу сделать случайной, при этом добавить шаг проверки матрицы на обратимостью.

**Обоснование**

Математическое обоснование алгоритма приведено в предыдущем пункте.

## Проверка алгоритма

In [7]:
# Разделим данные на признаки и целевой признак

features = insurance.drop('Страховые выплаты',axis=1)
target = insurance['Страховые выплаты']

# Разбивка на обучающую и валидационную выборки
features_train, features_valid, target_train, target_valid = train_test_split(features,
                                                                              target,
                                                                              test_size=0.25,
                                                                              random_state=112345
                                                                              )
# стандартизация
scaler = StandardScaler()
scaler.fit(features_train) 
features_train = scaler.transform(features_train)
features_valid = scaler.transform(features_valid)
        
# Проверка на соответсвие размеров выборок
print()
print('Размер валидационной выборки:')
print('features:',features_valid.shape)
print('target:',target_valid.shape)

print('Размер обучающей выборки:')
print('features:',features_train.shape)
print('target:',target_train.shape)


Размер валидационной выборки:
features: (1250, 4)
target: (1250,)
Размер обучающей выборки:
features: (3750, 4)
target: (3750,)


In [8]:
# Создадим случайную матрицу для преобразования признаков и проверим ее обратимость

MATRIX_SIZE = features.shape[1]

while True:    
    try:
        p_matrix = np.random.rand(MATRIX_SIZE,MATRIX_SIZE)
        np.linalg.inv(p_matrix)
    
    except np.linalg.LinAlgError:
        print("Создание другой случайной матрицы")
    
    else:
        print ('Случайнай обратимая матрица:')
        print(p_matrix)
        break

Случайнай обратимая матрица:
[[0.25111681 0.11292919 0.28784395 0.6184333 ]
 [0.42005457 0.06852652 0.84518013 0.59011899]
 [0.97585049 0.154339   0.74857894 0.88589847]
 [0.89942265 0.10040271 0.35091448 0.94658202]]


In [9]:
# Преобразуем признаки

features_train_transform = np.dot(features_train, p_matrix)
features_valid_transform = np.dot(features_valid, p_matrix)

In [10]:
# Функция обучения модели линейной регрессии и оценки ее качества

def prediction(features_train,target_train,features_valid,target_valid):
    model = LinearRegression()

    model.fit(features_train,target_train)
    prediction = model.predict(features_valid)

    r2 = r2_score(target_valid,prediction)

    return prediction,r2

In [11]:
# Обучим модель и вычислим оценку R2_score для исходных и преобразованных данных

# исходные данные
prediction_real,r2_real = prediction(features_train,
                                     target_train,
                                     features_valid,
                                     target_valid)

# преобразованные данные
prediction_transform,r2_transform = prediction(features_train_transform,
                                               target_train,
                                               features_valid_transform,
                                               target_valid)


In [12]:
# Сравним оценки
print('R2_score для исходных данных:\t\t',r2_real)
print('R2_score для преобразованных данных:\t',r2_transform)

R2_score для исходных данных:		 0.4087909565902036
R2_score для преобразованных данных:	 0.40879095659020337


**Вывод**

Результат метрики одинаковый для исходных и преоразованных данных. Соответвенно, защита данных клиентов никак не повлияла на результаты работы модели.

